In [1]:
import time
import numpy as np
import pandas as pd

EXECUTION_START_TIME = time.time()

time = pd.read_csv('data/Time.csv')
kospi = pd.read_csv('data/KOSPI.csv')

/home/tingyu/.local/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
# import
pd.options.mode.chained_assignment = None      # 解決會噴 warning 的狀況

from sklearn.impute import SimpleImputer       # 匯入填補缺失值的工具
from sklearn.preprocessing import LabelEncoder # 匯入 Label Encoder
from sklearn import preprocessing
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

In [3]:
print(time['date'].describe())

count            133
unique           133
top       2020-05-02
freq               1
Name: date, dtype: object


## 轉換日期資料
把日期資料型態轉換成python Timestamp

In [4]:
time["date"] = pd.to_datetime(time["date"], format='%Y-%m-%d')
kospi["Date"] = pd.to_datetime(kospi["Date"], format='%b %d, %Y')

In [5]:
print(kospi["Price"][0])
print(kospi["Date"][0])

kospi = kospi.iloc[::-1]
kospi = kospi.reset_index(drop=True)

print(kospi["Price"][0])
print(kospi["Date"][0])

2,181.87
2020-06-05 00:00:00
2,175.17
2020-01-02 00:00:00


## 轉換數值資料
把數值轉換成類型

In [6]:
for i in range(0, len(kospi.index)):
    kospi['Change %'][i] = 0 if kospi['Change %'][i][0] =='-' else 1

## 整理確診資料
將累積確診資料換算成每日確診人數

In [7]:
time["diff confirmed"] = 0
time["diff confirmed"][0] = time["confirmed"][0]
pre = time["confirmed"][0]

for i in range(1, len(time.index)):
    time["diff confirmed"][i] = time["confirmed"][i] - pre
    pre = time["confirmed"][i]

## 彙整資料
把確診和kospi資料合併

In [8]:
kospi["comfirmed_1"] = np.nan
kospi["comfirmed_2"] = np.nan
kospi["comfirmed_3"] = np.nan
kospi["groth_1"] = np.nan
kospi["groth_2"] = np.nan
kospi["groth_3"] = np.nan

for i in range(0, len(kospi.index)):
    if kospi["Date"][i] < time["date"][0]+datetime.timedelta(days=3):
        continue
    if kospi["Date"][i]-datetime.timedelta(days=1) > time["date"][len(time.index)-1]:
        continue
    if i-3<0:
        continue
    date = kospi["Date"][i]
    kospi["comfirmed_1"][i] = time['diff confirmed'][time["date"]==(date-datetime.timedelta(days=1))]
    kospi["comfirmed_2"][i] = time['diff confirmed'][time["date"]==(date-datetime.timedelta(days=2))]
    kospi["comfirmed_3"][i] = time['diff confirmed'][time["date"]==(date-datetime.timedelta(days=3))]
    kospi["groth_1"][i] = kospi['Change %'][i-1]
    kospi["groth_2"][i] = kospi['Change %'][i-2]
    kospi["groth_3"][i] = kospi['Change %'][i-3]


kospi = kospi.dropna()
kospi = kospi.reset_index(drop=True)

15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101


## 選取資料

In [9]:
x = kospi[["comfirmed_1", "comfirmed_2", "comfirmed_3"]]
y = kospi[["Change %"]]
date = kospi[["Date"]]

In [10]:
print(x.describe())

       comfirmed_1  comfirmed_2  comfirmed_3
count    87.000000    87.000000    87.000000
mean     86.022989    91.597701    93.356322
std     138.508918   154.880484   161.273708
min       0.000000     0.000000     0.000000
25%       8.000000     7.000000     7.000000
50%      27.000000    27.000000    25.000000
75%      99.000000   100.500000    99.000000
max     600.000000   813.000000   813.000000
